This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'Enter API key here'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [132]:
# First, import the relevant modules
!pip install requests
import requests
import math as m


In [41]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=' + API_KEY,
                params = {'start_date':'2019-10-23', 'end_date':'2019-10-23'})

In [42]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()['dataset']

{'id': 10095370,
 'dataset_code': 'AFX_X',
 'database_code': 'FSE',
 'name': 'Carl Zeiss Meditec (AFX_X)',
 'description': 'Stock Prices for Carl Zeiss Meditec (2019-10-10) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
 'refreshed_at': '2019-10-24T22:18:57.828Z',
 'newest_available_date': '2019-10-24',
 'oldest_available_date': '2000-06-07',
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'frequency': 'daily',
 'type': 'Time Series',
 'premium': False,
 'limit': None,
 'transform': None,
 'column_index': None,
 'start_date': '2019-10-23',
 'end_date': '2019-10-23',
 'data': [['2019-10-23',
   100.4,
   100.8,
   89.35,
   92.55,
   None,
   605001.0,
   56532324.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None,
 'database_id': 6129}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

****
## Task 1
 + Collect data for the year 2017

In [56]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=' + API_KEY,
                params = {'start_date':'2017-01-01', 'end_date':'2017-12-31'})

****
## Task 2
 + Convert to python dictionary

In [67]:
afx_data = []
for l in r.json()['dataset']['data']:
    afx_data.append(dict(zip(r.json()['dataset']['column_names'], l)))

****
## Task 3
 + Create a list of tuples (open price, date). Filter out 'None'.
 + Print the min and max values from the list along with their corresponding dates.

In [144]:
open_prices = [(d['Open'], d['Date']) for d in afx_data if d['Open']]

In [145]:
max_open = str(max(open_prices)[0]) + " on " + max(open_prices, key = lambda i : i[0])[1]
print(max_open)

53.11 on 2017-12-14


In [146]:
min_open = str(min(open_prices)[0]) + " on " + min(open_prices, key = lambda i : i[0])[1]
print(min_open)

34.0 on 2017-01-24


****
## Task 4
 + Create a list of tuples (daily price change, date). by subtracting the Low from the High
 + Print the max of the list along with its corresponding date

In [147]:
daily_high_low = [(d['High'] - d['Low'], d['Date']) for d in afx_data if d['Open'] and d['Close']]

In [150]:
largest_swing = str(max(daily_high_low)[0]) + ' on ' + max(daily_high_low, key = lambda i : i[0])[1]
print(largest_swing)

2.8100000000000023 on 2017-05-11


****
## Task 5
 + Make a new API call where we request the change from the previous days values
 + Print the max absolute value of the closing price changes

In [97]:
r2 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=' + API_KEY,
                params = {'start_date':'2017-01-01', 'end_date':'2017-12-31', 'transform':'diff'})

In [106]:
afx_data2 = []
for l in r2.json()['dataset']['data']:
    afx_data2.append(dict(zip(r2.json()['dataset']['column_names'], l)))

In [108]:
closing_price_changes = [d['Close'] for d in afx_data2 if d['Close']]

In [115]:
print(max(closing_price_changes, key=abs))

-2.56


****
## Task 6
 + Make a list of daily volumes from the original API call
 + Calculate the average of the daily volumes

In [123]:
daily_volumes = [d['Traded Volume'] for d in afx_data if d['Traded Volume']]

In [124]:
print(sum(daily_volumes) / len(daily_volumes))

89124.33725490196


****
## Task 7
 + Define a function that returns either the middle number or the average of the two middle numbers of an ordered list
 + Call the function on the ordered list of daily volumes

In [135]:
def give_median(x):
    """Finds the median of an ordered list"""
    if len(x) // 2 == 0:
        median = (x[(len(x)/2)] + x[(len(x)/2) + 1]) / 2
    else:
        median = x[m.ceil(len(x)/2)]
    
    return median

In [136]:
give_median(daily_volumes)

34640.0